# Home Credit Modeling

In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import iqr, randint, uniform
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_objective
import pickle
import gc

%load_ext autotime

gc.enable()
np.random.seed(235)

path = "/Users/dsaxton/home_credit_default/"

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 5.06 ms


#### Load the data

In [140]:
df = pd.read_csv(path + "train_pruned.csv")

time: 12.4 s


In [141]:
sk_id_curr = df.pop("SK_ID_CURR")
y = df.pop("TARGET")

time: 90.5 ms


#### Select features

In [ ]:
clf = lgb.LGBMClassifier(n_estimators=1000, num_leaves=23, subsample=0.5)
clf.fit(df, y)

lgb_cols = df.columns[clf.feature_importances_ > 0]
len(lgb_cols)

In [ ]:
var_imp = pd.DataFrame({"Feature": df.columns, "Importance": clf.feature_importances_})[["Feature", "Importance"]].sort_values("Importance", ascending=False)

#### View important features

In [ ]:
var_imp.head(20)

#### LightGBM CV

In [139]:
params = {"n_estimators": 20000, 
          "num_leaves": 500, 
          "min_data_in_leaf": 2000,
          "learning_rate": 0.003, 
          "bagging_fraction": 0.5, 
          "bagging_freq": 1, 
          "feature_fraction": 0.5, 
          "lambda_l2": 1}

lgb_data = lgb.Dataset(data=df, 
                       label=y)

cv_result = lgb.cv(params=params, 
                   train_set=lgb_data, 
                   nfold=5, 
                   metrics="auc", 
                   early_stopping_rounds=200, 
                   stratified=True, 
                   shuffle=True, 
                   verbose_eval=100, 
                   show_stdv=True, 
                   seed=2357)

cv_result = pd.DataFrame(cv_result)

/Users/dsaxton/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's auc: 0.758237 + 0.00301429
[200]	cv_agg's auc: 0.760392 + 0.00299338
[300]	cv_agg's auc: 0.762976 + 0.00292398
[400]	cv_agg's auc: 0.765112 + 0.00280283
[500]	cv_agg's auc: 0.767004 + 0.00271434
[600]	cv_agg's auc: 0.768901 + 0.00258913
[700]	cv_agg's auc: 0.770513 + 0.00251051
[800]	cv_agg's auc: 0.772057 + 0.00243223
[900]	cv_agg's auc: 0.773453 + 0.00236621
[1000]	cv_agg's auc: 0.774693 + 0.00233021
[1100]	cv_agg's auc: 0.775823 + 0.00227372
[1200]	cv_agg's auc: 0.77688 + 0.00223688
[1300]	cv_agg's auc: 0.777806 + 0.00221203
[1400]	cv_agg's auc: 0.77865 + 0.0021778
[1500]	cv_agg's auc: 0.779462 + 0.00217178
[1600]	cv_agg's auc: 0.78019 + 0.00215081
[1700]	cv_agg's auc: 0.78086 + 0.0021444
[1800]	cv_agg's auc: 0.781468 + 0.0021403
[1900]	cv_agg's auc: 0.782042 + 0.00214039
[2000]	cv_agg's auc: 0.782578 + 0.00215126
[2100]	cv_agg's auc: 0.783072 + 0.00217638
[2200]	cv_agg's auc: 0.783537 + 0.0021833
[2300]	cv_agg's auc: 0.783992 + 0.00219141
[2400]	cv_agg's auc: 0.78441

KeyboardInterrupt: 

time: 1h 33min 53s


In [ ]:
cv_result.tail()

```
params = {"n_estimators": 20000, 
          "num_leaves": 500, 
          "min_data_in_leaf": 2000,
          "learning_rate": 0.003, 
          "bagging_fraction": 0.5, 
          "feature_fraction": 0.5, 
          "lambda_l2": 1}
```

#### Fit chosen model

CV mean:  0.791015

In [78]:
lgb_model = lgb.LGBMClassifier(n_estimators=2520, 
                         num_leaves=233, 
                         min_data_in_leaf=1000,
                         learning_rate=0.005, 
                         bagging_fraction=0.5, 
                         feature_fraction=0.5, 
                         lambda_l2=0.8)

lgb_model.fit(df[lgb_cols], y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
        learning_rate=0.0025, max_depth=10, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=6876,
        n_jobs=-1, num_leaves=113, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.8, silent=True, subsample=0.5,
        subsample_for_bin=200000, subsample_freq=1)

time: 30min 31s


#### Score test data and generate submission

In [79]:
df = pd.read_csv(path + "test.csv")

time: 4.26 s


In [80]:
df["NAME_INCOME_TYPE_Maternity_leave"] = 0

time: 10 ms


In [81]:
submission = pd.DataFrame({"SK_ID_CURR": df["SK_ID_CURR"], 
                           "TARGET": lgb_model.predict_proba(df[lgb_cols])[:,1]})

submission.head()

,SK_ID_CURR,TARGET
0,100001,0.033505
1,100005,0.123829
2,100013,0.026391
3,100028,0.033826
4,100038,0.170974


time: 26.3 s


In [82]:
submission.to_csv(path + "submission.csv", index=False)

time: 176 ms
